In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


./data\data.csv


In [3]:
data = pd.read_csv('./data/data.csv')
# Raw Data as read by Pandas
print (data)

      MONTH STORECODE  DAY BILL_ID  BILL_AMT  QTY  VALUE  PRICE  \
0        M1        N1    4    T375     225.0  1.0  225.0  225.0   
1        M1        N1    4    T379      95.0  1.0   95.0   95.0   
2        M1        N1    4    T381      10.0  1.0   10.0   10.0   
3        M1        N1    4    T382     108.0  1.0  108.0  108.0   
4        M1        N1    4    T384      19.0  1.0   19.0   19.0   
...     ...       ...  ...     ...       ...  ...    ...    ...   
26980    M3       N10   31   T5999      10.0  1.0   10.0   10.0   
26981    M3       N10   31   T6007      88.0  1.0   10.0   10.0   
26982    M3       N10   31   T6007      88.0  2.0   20.0   10.0   
26983    M3       N10   31   T6007      88.0  1.0   58.0   58.0   
26984    M3       N10   31   T6013      15.0  1.0   15.0   15.0   

                              GRP                     SGRP  \
0            BUTTER MARGR  (4/94)                   BUTTER   
1         CONFECTIONERY - ECLAIRS  CONFECTIONERY - ECLAIRS   
2        

In [4]:
print('There are {} distinct data at Group level'.format(data['GRP'].nunique()))
print('There are {} distinct data at sub-Group level'.format(data['SGRP'].nunique()))
print('There are {} distinct data at sub-sub-Group level'.format(data['SSGRP'].nunique()))
print('There are {} distinct data at Company level'.format(data['CMP'].nunique()))
print('There are {} distinct data at Mother-Brand level'.format(data['MBRD'].nunique()))
print('There are {} distinct data at Brand level'.format(data['BRD'].nunique()))

There are 80 distinct data at Group level
There are 174 distinct data at sub-Group level
There are 232 distinct data at sub-sub-Group level
There are 354 distinct data at Company level
There are 643 distinct data at Mother-Brand level
There are 1315 distinct data at Brand level


In [5]:
data_N1 = data[data['STORECODE'] == 'N1']
print(data_N1.shape)

(4583, 14)


In [17]:
data_N1 = data_N1[['BILL_ID', 'QTY', 'GRP']]
print(data_N1)

      BILL_ID  QTY                             GRP
0        T375  1.0            BUTTER MARGR  (4/94)
1        T379  1.0         CONFECTIONERY - ECLAIRS
2        T381  1.0                       CHOCOLATE
3        T382  1.0                    PACKAGED TEA
4        T384  1.0                ALL IODISED SALT
...       ...  ...                             ...
17468   T9251  1.0      BISCUITS - CORE & NON CORE
17469   T9251  1.0             VERMICELLI & NOODLE
17470   T9251  1.0            TOILET SOAPS (04/00)
17471   T9255  1.0      CONFECTIONERY - TOFFEE/HBC
17472   T9255  2.0  MOSQUITO REPELLANTS CTG (8/93)

[4583 rows x 3 columns]


In [18]:
basket = data_N1.groupby(['BILL_ID', 'GRP'])['QTY'].sum().unstack().reset_index().fillna(0).set_index('BILL_ID')
print (basket)

GRP      AGARBATTI & DHOOPBATTI  ALL AIR FRESHNERS(01/03)  ALL IODISED SALT  \
BILL_ID                                                                       
T1000                       0.0                       0.0               0.0   
T1001                       0.0                       0.0               0.0   
T1003                       0.0                       0.0               0.0   
T1004                       0.0                       0.0               0.0   
T1005                       0.0                       0.0               0.0   
...                         ...                       ...               ...   
T988                        0.0                       0.0               0.0   
T989                        0.0                       0.0               0.0   
T991                        0.0                       0.0               0.0   
T996                        0.0                       0.0               0.0   
T998                        0.0                     

In [20]:
def encode_unit(x):
    if x <= 0:
        return 0
    if x >= 0:
        return 1
    
encoded_data = basket.applymap(encode_unit)
print (encoded_data)

GRP      AGARBATTI & DHOOPBATTI  ALL AIR FRESHNERS(01/03)  ALL IODISED SALT  \
BILL_ID                                                                       
T1000                         0                         0                 0   
T1001                         0                         0                 0   
T1003                         0                         0                 0   
T1004                         0                         0                 0   
T1005                         0                         0                 0   
...                         ...                       ...               ...   
T988                          0                         0                 0   
T989                          0                         0                 0   
T991                          0                         0                 0   
T996                          0                         0                 0   
T998                          0                     

In [21]:
frequent_items = apriori(encoded_data, min_support = 0.01, use_colnames = True)
print (frequent_items)

     support                                           itemsets
0   0.019577                                 (ALL IODISED SALT)
1   0.010180                                        (BATTERIES)
2   0.012921                                        (BEVERAGES)
3   0.284652                       (BISCUITS - CORE & NON CORE)
4   0.038763                                (BREAKFAST CEREALS)
5   0.024667                             (BUTTER MARGR  (4/94))
6   0.014096                             (CHEESE        (4/94))
7   0.205168                                        (CHOCOLATE)
8   0.014879                     (CONFECTIONERY - LOLLIPOP/FMR)
9   0.077525                       (CONFECTIONERY - TOFFEE/HBC)
10  0.023101                        (CONFECTIONERY - TOTAL GUM)
11  0.016053                             (DETERGENT CAKES/BARS)
12  0.022709                             (DIAPERS       (8/93))
13  0.023101                                  (KETCHUPS/SAUCES)
14  0.011746                            

In [25]:
rules = association_rules(frequent_items, metric = 'lift', min_threshold = 0)
print(rules)

                     antecedents                   consequents  \
0            (BREAKFAST CEREALS)  (BISCUITS - CORE & NON CORE)   
1   (BISCUITS - CORE & NON CORE)           (BREAKFAST CEREALS)   
2                    (CHOCOLATE)  (BISCUITS - CORE & NON CORE)   
3   (BISCUITS - CORE & NON CORE)                   (CHOCOLATE)   
4   (BISCUITS - CORE & NON CORE)  (CONFECTIONERY - TOFFEE/HBC)   
5   (CONFECTIONERY - TOFFEE/HBC)  (BISCUITS - CORE & NON CORE)   
6          (SALTY SNACKS (2/97))  (BISCUITS - CORE & NON CORE)   
7   (BISCUITS - CORE & NON CORE)         (SALTY SNACKS (2/97))   
8          (VERMICELLI & NOODLE)  (BISCUITS - CORE & NON CORE)   
9   (BISCUITS - CORE & NON CORE)         (VERMICELLI & NOODLE)   
10                   (CHOCOLATE)  (CONFECTIONERY - TOFFEE/HBC)   
11  (CONFECTIONERY - TOFFEE/HBC)                   (CHOCOLATE)   
12         (SALTY SNACKS (2/97))                   (CHOCOLATE)   
13                   (CHOCOLATE)         (SALTY SNACKS (2/97))   
14        

In [24]:
store = list(data['STORECODE'].unique())
print(store)

['N1', 'N2', 'N3', 'N4', 'N5', 'N6', 'N7', 'N8', 'N9', 'N10']


In [28]:
for i in range(0,len(store)):
    data_N = data[data['STORECODE'] == store[i]]
    data_N = data_N[['BILL_ID', 'QTY', 'GRP']]
    basket = data_N.groupby(['BILL_ID', 'GRP'])['QTY'].sum().unstack().reset_index().fillna(0).set_index('BILL_ID')
    
    def encode_unit(x):
        if x <= 0:
            return 0
        if x >= 0:
            return 1
    
    encoded_data = basket.applymap(encode_unit)
    frequent_items = apriori(encoded_data, min_support = 0.01, use_colnames = True)
    
    rules = association_rules(frequent_items, metric = 'lift', min_threshold = 0)
    print(rules)
    rules.to_csv('./Output/AssociationRuleforstore{}.csv'.format(store[i]))

                     antecedents                   consequents  \
0            (BREAKFAST CEREALS)  (BISCUITS - CORE & NON CORE)   
1   (BISCUITS - CORE & NON CORE)           (BREAKFAST CEREALS)   
2                    (CHOCOLATE)  (BISCUITS - CORE & NON CORE)   
3   (BISCUITS - CORE & NON CORE)                   (CHOCOLATE)   
4   (BISCUITS - CORE & NON CORE)  (CONFECTIONERY - TOFFEE/HBC)   
5   (CONFECTIONERY - TOFFEE/HBC)  (BISCUITS - CORE & NON CORE)   
6          (SALTY SNACKS (2/97))  (BISCUITS - CORE & NON CORE)   
7   (BISCUITS - CORE & NON CORE)         (SALTY SNACKS (2/97))   
8          (VERMICELLI & NOODLE)  (BISCUITS - CORE & NON CORE)   
9   (BISCUITS - CORE & NON CORE)         (VERMICELLI & NOODLE)   
10                   (CHOCOLATE)  (CONFECTIONERY - TOFFEE/HBC)   
11  (CONFECTIONERY - TOFFEE/HBC)                   (CHOCOLATE)   
12         (SALTY SNACKS (2/97))                   (CHOCOLATE)   
13                   (CHOCOLATE)         (SALTY SNACKS (2/97))   
14        

                     antecedents                   consequents  \
0         (BUTTER MARGR  (4/94))  (BISCUITS - CORE & NON CORE)   
1   (BISCUITS - CORE & NON CORE)        (BUTTER MARGR  (4/94))   
2                    (CHOCOLATE)  (BISCUITS - CORE & NON CORE)   
3   (BISCUITS - CORE & NON CORE)                   (CHOCOLATE)   
4   (BISCUITS - CORE & NON CORE)          (CLEANERS - UTENSIL)   
..                           ...                           ...   
57     (WASHING POWDERS/LIQUIDS)        (TOILET SOAPS (04/00))   
58     (WASHING POWDERS/LIQUIDS)                (TOOTH PASTES)   
59                (TOOTH PASTES)     (WASHING POWDERS/LIQUIDS)   
60     (WASHING POWDERS/LIQUIDS)         (VERMICELLI & NOODLE)   
61         (VERMICELLI & NOODLE)     (WASHING POWDERS/LIQUIDS)   

    antecedent support  consequent support   support  confidence      lift  \
0             0.066279            0.255814  0.011628    0.175439  0.685805   
1             0.255814            0.066279  0.01162

                        antecedents                      consequents  \
0      (BISCUITS - CORE & NON CORE)  (REFINED EDIBLE OILS-ALL PACKS)   
1   (REFINED EDIBLE OILS-ALL PACKS)     (BISCUITS - CORE & NON CORE)   
2             (SALTY SNACKS (2/97))     (BISCUITS - CORE & NON CORE)   
3      (BISCUITS - CORE & NON CORE)            (SALTY SNACKS (2/97))   
4                  (SPICES (03/04))     (BISCUITS - CORE & NON CORE)   
5      (BISCUITS - CORE & NON CORE)                 (SPICES (03/04))   
6      (BISCUITS - CORE & NON CORE)            (VERMICELLI & NOODLE)   
7             (VERMICELLI & NOODLE)     (BISCUITS - CORE & NON CORE)   
8   (REFINED EDIBLE OILS-ALL PACKS)                   (PACKAGED TEA)   
9                    (PACKAGED TEA)  (REFINED EDIBLE OILS-ALL PACKS)   
10                 (SPICES (03/04))  (REFINED EDIBLE OILS-ALL PACKS)   
11  (REFINED EDIBLE OILS-ALL PACKS)                 (SPICES (03/04))   
12           (TOILET SOAPS (04/00))  (REFINED EDIBLE OILS-ALL PA